In [ ]:
# default_exp image.inference

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import asyncio
import numpy as np
from fastai.basic_train import load_learner
from fastai import vision

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Helper Methods

In [ ]:
# export


async def setup_learner(path):
    """
    Setup the learner with weights.
    `path`: The path to the folder containing the exported model file.
    """
    try:
        return load_learner(path.parent, path.name)
    except RuntimeError as e:
        if len(e.args) > 0 and 'CPU-only machine' in e.args[0]:
            print(e)
            messages = [
                "This model was trained with an old version of fastai and will not work in a CPU environment.",
                "Please update the fastai library in your training environment and export your model again.",
            ]
            message = "\n\n{}".format("\n\n".join(messages))
            raise RuntimeError(message)
        else:
            raise

In [ ]:
# export


def fix_odd_sides(img):
    """
    Fix odd image sizes.
    Odd image sizes can not be divided by 2.
    `img`: The image to fix as fastai image.
    return: the image reference.
    """
    if (list(img.size)[0] % 2) != 0:
        img = vision.crop_pad(img, size=(list(img.size)[0] + 1, list(img.size)[1]), padding_mode='reflection')

    if (list(img.size)[1] % 2) != 0:
        img = vision.crop_pad(img, size=(list(img.size)[0], list(img.size)[1] + 1), padding_mode='reflection')

    return img

In [ ]:
# export


class Inference:
    """
    Base class for model inference.
    """

    def __init__(self, path):
        """
        Initialize an new inference.

        `path`: path to the model to load
        """
        self.path = path
        loop = asyncio.get_event_loop()
        tasks = [asyncio.ensure_future(setup_learner(path))]
        self.learner = loop.run_until_complete(asyncio.gather(*tasks))[0]
        loop.close()

    def destroy(self):
        """
        Remove the current loaded model from memory.
        """
        if self.learner:
            self.learner.destroy()

    def predict(self, x):
        """
        Predicts an image.

        `x`: the input to predict as pytorch tensor.
        """
        pass

In [ ]:
# export


class ClassificationInference(Inference):
    """
    Inference of a classification model.
    Supports single label and multi label classification.
    """

    def predict(self, x):
        """
        Predicts an image.

        `x`: the input to predict as pytorch tensor.
        returns: a tuple containing the prediction, labels and probability distribution
        """
        pred, labels, probs = self.learner.predict(x)
        return pred, labels, probs

In [ ]:
# export


class SegmentationInference(Inference):
    """
    Inference of a segmentation model.
    """

    def predict(self, x):
        """
        Predicts an image.

        `x`: the input to predict as pytorch tensor.
        returns: a tuple containing the segmentation mask, labels and probability distribution
        """
        x = fix_odd_sides(x)
        pred, labels, probs = self.learner.predict(x)
        mask = np.asarray(vision.image2np(pred.data), dtype=np.uint8)
        return mask, labels, probs

In [ ]:
# export
